# 🎓 Fine-Tuning Mistral 7B - Philosophes (LoRA)

**Objectif :** Fine-tuner Mistral 7B sur 1200 exemples de schèmes logiques philosophiques

**GPU Optimal :** A100 40GB (30-45 min) | V100 16GB (1h-1h30) | T4 15GB (2-3h)

**Config :**
- Modèle : `mistralai/Mistral-7B-Instruct-v0.3`
- Méthode : QLoRA (4-bit) + LoRA (r=64, alpha=128)
- Dataset : 1200 exemples (300 base + 900 augmentés)
- Epochs : 3
- Batch size : 8 (A100) | 4 (V100/T4)

## 1️⃣ Setup - Installation & Vérifications

In [ ]:
# Installation des dépendances optimisées (versions récentes)
# Note: Versions spécifiques pour éviter conflits avec torchaudio/torchvision
print("📦 Installation des packages (peut prendre 2-3 minutes)...\n")

!pip install -U \
    torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 \
    transformers>=4.40.0 \
    peft>=0.10.0 \
    bitsandbytes>=0.43.0 \
    accelerate>=0.28.0 \
    trl>=0.8.0 \
    datasets>=2.18.0 \
    huggingface_hub>=0.22.0

print("\n✅ Installation terminée !")

In [ ]:
# Vérifier le GPU disponible
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
    
    # Déterminer config optimale selon GPU
    if "A100" in gpu_name and gpu_memory > 35:
        print("\n🚀 GPU OPTIMAL détecté: A100 40GB")
        BATCH_SIZE = 8
        GRADIENT_ACCUM = 4
    elif "V100" in gpu_name or ("A100" in gpu_name and gpu_memory < 20):
        print("\n✅ GPU EXCELLENT détecté: V100/A100-16GB")
        BATCH_SIZE = 4
        GRADIENT_ACCUM = 8
    else:
        print("\n🟡 GPU STANDARD détecté: T4")
        BATCH_SIZE = 2
        GRADIENT_ACCUM = 16
    
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Gradient accumulation: {GRADIENT_ACCUM}")
    print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM}")
else:
    print("⚠️ CUDA non disponible - Training sur CPU (très lent)")
    BATCH_SIZE = 1
    GRADIENT_ACCUM = 32

## 2️⃣ Configuration - Authentification HF & Datasets

In [ ]:
# Authentification Hugging Face (pour push du modèle)
from huggingface_hub import login
from google.colab import userdata

# Option 1: Utiliser les secrets Colab (RECOMMANDÉ)
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ Token HF récupéré depuis Colab Secrets")
except:
    # Option 2: Saisir manuellement si secret non défini
    print("⚠️ Token HF_TOKEN non trouvé dans Colab Secrets")
    print("📝 Pour configurer le secret Colab:")
    print("   1. Cliquer sur l'icône 🔑 (clé) dans la barre latérale gauche")
    print("   2. Ajouter un nouveau secret: Nom = HF_TOKEN, Valeur = votre token HF")
    print("   3. Activer l'accès pour ce notebook")
    print("   4. Redémarrer ce notebook\n")
    
    # Fallback: saisir directement
    from getpass import getpass
    HF_TOKEN = getpass("Entrez votre token HF (obtenir sur https://huggingface.co/settings/tokens): ")

# Login avec le token
login(token=HF_TOKEN)

print("✅ Authentification HF réussie")

In [ ]:
# Charger les datasets depuis fichiers locaux
# ⚠️ UPLOADER LES FICHIERS DANS COLAB :
# - schemes_levelA_base.jsonl
# - schemes_levelA_augmented.jsonl

from datasets import load_dataset, concatenate_datasets

# Charger les datasets
dataset_base = load_dataset('json', data_files='schemes_levelA_base.jsonl', split='train')
dataset_augmented = load_dataset('json', data_files='schemes_levelA_augmented.jsonl', split='train')

# Combiner les datasets
dataset_full = concatenate_datasets([dataset_base, dataset_augmented])

print(f"✅ Dataset chargé: {len(dataset_full)} exemples")
print(f"   - Base: {len(dataset_base)} exemples")
print(f"   - Augmentés: {len(dataset_augmented)} exemples")

# Afficher un exemple
print("\n📝 Exemple de données:")
print(dataset_full[0]['messages'])

In [ ]:
# Split train/validation (95/5)
dataset_split = dataset_full.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print(f"✅ Split réalisé:")
print(f"   - Train: {len(train_dataset)} exemples")
print(f"   - Validation: {len(eval_dataset)} exemples")

# Formater les datasets pour SFTTrainer (ajouter champ "text")
def format_chat_template(example):
    """Formate les messages au format ChatML et ajoute champ 'text'"""
    messages = example['messages']
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

# Appliquer le formatage
train_dataset = train_dataset.map(format_chat_template, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(format_chat_template, remove_columns=eval_dataset.column_names)

print(f"\n✅ Datasets formatés avec champ 'text'")
print(f"   Exemple de texte formaté:")
print(train_dataset[0]['text'][:200] + "...")

## 3️⃣ Modèle - Configuration QLoRA + LoRA

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

# Configuration quantization 4-bit (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",           # NormalFloat4 (meilleure précision)
    bnb_4bit_compute_dtype=torch.bfloat16,  # bfloat16 optimal sur A100
    bnb_4bit_use_double_quant=True,      # Double quantization (économie VRAM)
)

print("📥 Chargement du modèle Mistral 7B...")

# Charger le modèle base en 4-bit
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Modèle chargé en 4-bit")

# Préparer le modèle pour training k-bit
model = prepare_model_for_kbit_training(model)

print("✅ Modèle préparé pour k-bit training")

In [ ]:
# Configuration LoRA (rang 64 pour qualité maximale)
lora_config = LoraConfig(
    r=64,                              # Rang LoRA (64 = haute qualité)
    lora_alpha=128,                    # Alpha = 2 * r (règle empirique)
    lora_dropout=0.05,                 # Dropout léger
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[                   # Tous les modules attention + MLP
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

# Appliquer LoRA au modèle
model = get_peft_model(model, lora_config)

# Afficher les paramètres entraînables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA appliqué (r={lora_config.r}, alpha={lora_config.lora_alpha})")
print(f"   Paramètres entraînables: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"   Paramètres totaux: {total_params:,}")

## 4️⃣ Training - Configuration & Lancement

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Configuration du training
training_args = TrainingArguments(
    # Output
    output_dir="./mistral-7b-philosophes-lora",
    
    # Batch size (ajusté selon GPU)
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM,
    
    # Learning rate
    learning_rate=2e-4,              # Optimal pour LoRA
    lr_scheduler_type="cosine",      # Cosine avec warmup
    warmup_ratio=0.03,               # 3% warmup
    
    # Epochs
    num_train_epochs=3,              # 3 epochs pour éviter overfitting
    
    # Optimisation
    optim="paged_adamw_8bit",        # AdamW 8-bit (économie VRAM)
    bf16=True,                        # bfloat16 sur A100
    fp16=False,
    
    # Gradient clipping
    max_grad_norm=0.3,
    
    # Logging & Evaluation
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    save_total_limit=3,
    
    # Misc
    report_to="none",                # Pas de W&B/Tensorboard
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

print("✅ Configuration training créée")
print(f"   Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM}")
print(f"   Total steps: ~{len(train_dataset) * 3 // (BATCH_SIZE * GRADIENT_ACCUM)}")

In [ ]:
# Créer le trainer (API simplifiée trl>=0.8.0)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
)

print("✅ Trainer créé et prêt")

In [ ]:
# 🚀 LANCER LE TRAINING
print("🚀 Démarrage du training...\n")

# Afficher temps estimé
if "A100" in torch.cuda.get_device_name(0) and torch.cuda.get_device_properties(0).total_memory > 35e9:
    print("⏱️ Temps estimé: 30-45 minutes (A100 40GB)")
elif "V100" in torch.cuda.get_device_name(0):
    print("⏱️ Temps estimé: 1h-1h30 (V100)")
else:
    print("⏱️ Temps estimé: 2-3h (T4)")

print("\n" + "="*60)

# Training
trainer.train()

print("\n" + "="*60)
print("✅ Training terminé !")

## 5️⃣ Export - Sauvegarde & Push vers HF Hub

In [ ]:
# Sauvegarder le modèle localement
output_dir = "./mistral-7b-philosophes-lora-final"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Modèle sauvegardé dans: {output_dir}")

# Afficher la taille du LoRA
import os
lora_size = sum(os.path.getsize(os.path.join(output_dir, f)) for f in os.listdir(output_dir)) / 1024**2
print(f"   Taille LoRA: {lora_size:.1f} MB")

In [ ]:
# Push vers HF Space 3_PHI (RECOMMANDÉ - mise à jour directe du Space)
SPACE_REPO = "FJDaz/3_PHI"

print(f"📤 Push du LoRA vers Space HF: {SPACE_REPO}...")
print("⚠️ Le LoRA sera ajouté au Space existant (ne remplace pas le modèle base)")

# Créer un sous-dossier "lora" dans le Space pour le LoRA
trainer.model.push_to_hub(
    SPACE_REPO,
    use_auth_token=HF_TOKEN,
    commit_message="Add Mistral 7B LoRA fine-tuned on 1200 philosophy schemas (r=64)",
    subfolder="lora",  # Push dans un sous-dossier pour ne pas écraser le Space
)

tokenizer.push_to_hub(
    SPACE_REPO,
    use_auth_token=HF_TOKEN,
    subfolder="lora",
)

print(f"✅ LoRA pushé sur Space: https://huggingface.co/spaces/{SPACE_REPO}")
print(f"   Emplacement: https://huggingface.co/spaces/{SPACE_REPO}/tree/main/lora")
print("\n💡 Pour utiliser le LoRA dans le Space:")
print("   1. Modifier app.py pour charger depuis './lora/'")
print("   2. PeftModel.from_pretrained(model, './lora/')")

## 6️⃣ Test - Inférence Rapide

In [ ]:
# Test rapide du modèle fine-tuné
import torch

print("📝 Test 1: Modus Ponens spinoziste")

test_prompt = """Schème : Modus Ponens
Contexte : Si l'homme ignore les causes de ses passions, il est en servitude. Or l'élève ignore les causes de ses passions.
Applique le schème :"""

print(f"Prompt: {test_prompt}")
print("\n" + "="*60)

messages = [
    {"role": "system", "content": "Tu es un tuteur philosophique maîtrisant les schèmes logiques."},
    {"role": "user", "content": test_prompt}
]

# Formater avec le tokenizer
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# Utiliser autocast pour gérer automatiquement les types mixtes (BFloat16/Float32)
print("🔧 Génération avec torch.autocast(bfloat16)...")
with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
        )

# Décoder la réponse
response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
print(response)

print("\n" + "="*60)

In [ ]:
# Test 2: Identité spinoziste
test_prompt_2 = """Schème : Identité
Contexte : Dieu = Nature. Or la Nature est nécessaire.
Applique le schème :"""

print("📝 Test 2: Identité spinoziste")
print(f"Prompt: {test_prompt_2}")
print("\n" + "="*60)

messages = [
    {"role": "system", "content": "Tu es un tuteur philosophique maîtrisant les schèmes logiques."},
    {"role": "user", "content": test_prompt_2}
]

output = pipe(messages)
print(output[0]['generated_text'][-1]['content'])

print("\n" + "="*60)
print("✅ Tests terminés !")

## 7️⃣ Download - Télécharger le LoRA Localement

In [ ]:
# Zipper le modèle pour téléchargement
!zip -r mistral-7b-philosophes-lora-final.zip ./mistral-7b-philosophes-lora-final/

print("✅ Archive créée: mistral-7b-philosophes-lora-final.zip")
print("📥 Téléchargez le fichier depuis l'explorateur Colab (gauche)")

---

## ✅ Checklist Finale

- [ ] Training terminé sans erreur
- [ ] Eval loss < 0.6 (validation)
- [ ] Tests d'inférence corrects (application schèmes)
- [ ] Modèle pushé sur HF Hub OU téléchargé localement
- [ ] Taille LoRA ~250-350 MB (r=64)

## 🎯 Prochaines Étapes

1. **Benchmarks complets** : Tester sur 30 questions (10 par philosophe)
2. **Test CPU** : Charger en 4-bit sur CPU, mesurer latence
3. **Déploiement HF Space** : Créer Space CPU gratuit avec ce LoRA
4. **Comparaison** : vs Mistral 7B base (sans LoRA) + prompts système

---

**Créé le :** 20 novembre 2025  
**Auteur :** Claude Code  
**Modèle :** Mistral 7B Instruct v0.3 + LoRA (r=64, alpha=128)  
**Dataset :** 1200 exemples schèmes philosophiques (Spinoza, Bergson, Kant)